# 🚗 MPG 자동차 연비 데이터 분석
## Scikit-Learn Train-Test Split 활용 실습

**목표**: MPG 데이터셋을 활용하여 다양한 데이터 분할 기법을 학습하고 실습합니다.

**데이터**: mpg.csv - 자동차 연비 및 제원 정보
- **manufacturer**: 제조사
- **model**: 모델명
- **displ**: 배기량 (L)
- **year**: 연식
- **cyl**: 실린더 수
- **trans**: 변속기 타입
- **drv**: 구동방식 (f=전륜, r=후륜, 4=사륜)
- **cty**: 시내 연비 (mpg)
- **hwy**: 고속도로 연비 (mpg)
- **fl**: 연료 타입
- **category**: 차량 카테고리

## 📚 Section 1: 환경 설정 및 라이브러리 Import

In [2]:
# 필수 라이브러리 Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Scikit-Learn 모듈들
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

# 한글 폰트 설정
def setup_korean_fonts():
    """한글 폰트를 자동으로 설정하는 함수"""
    import platform
    import matplotlib.font_manager as fm
    
    print("🔤 한글 폰트 설정 중...")
    
    # 시스템별 한글 폰트 리스트
    if platform.system() == 'Windows':
        font_candidates = ['Malgun Gothic', 'Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
    elif platform.system() == 'Darwin':  # macOS
        font_candidates = ['AppleGothic', 'Nanum Gothic', 'Arial Unicode MS']
    else:  # Linux
        font_candidates = ['Nanum Gothic', 'DejaVu Sans', 'Liberation Sans']
    
    # 사용 가능한 폰트 찾기
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    print(f"📝 시스템 폰트 수: {len(available_fonts)}개")
    
    selected_font = None
    for font in font_candidates:
        if font in available_fonts:
            selected_font = font
            break
    
    if selected_font:
        plt.rcParams['font.family'] = selected_font
        plt.rcParams['axes.unicode_minus'] = False  # 마이너스 부호 깨짐 방지
        print(f"✅ 선택된 한글 폰트: {selected_font}")
        return selected_font
    else:
        print("⚠️ 한글 폰트를 찾을 수 없어 기본 폰트를 사용합니다.")
        return "DejaVu Sans"

# 한글 폰트 설정 실행
font_name = setup_korean_fonts()
print(f"✅ 한글 폰트 설정 완료: {font_name}")

# 기본 설정
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다!")
print(f"📊 Pandas Version: {pd.__version__}")
print(f"🔢 NumPy Version: {np.__version__}")
print(f"🤖 Scikit-Learn 분할 모듈들이 준비되었습니다!")
print(f"📈 Matplotlib & Seaborn 시각화 준비완료!")
print(f"🔤 한글 폰트: {font_name}")

print("\n🚗 MPG 자동차 연비 데이터 분석을 시작하겠습니다!")

🔤 한글 폰트 설정 중...
📝 시스템 폰트 수: 505개
✅ 선택된 한글 폰트: Malgun Gothic
✅ 한글 폰트 설정 완료: Malgun Gothic
✅ 모든 라이브러리가 성공적으로 로드되었습니다!
📊 Pandas Version: 2.3.3
🔢 NumPy Version: 2.2.6
🤖 Scikit-Learn 분할 모듈들이 준비되었습니다!
📈 Matplotlib & Seaborn 시각화 준비완료!
🔤 한글 폰트: Malgun Gothic

🚗 MPG 자동차 연비 데이터 분석을 시작하겠습니다!


## 📊 실습1: MPG 데이터 로드 및 데이터프레임 형식 저장

In [3]:
# 🚗 MPG 자동차 연비 데이터 로드
print("🔄 MPG 자동차 연비 데이터를 로드하고 있습니다...")

try:
    # MPG CSV 파일 로드
    mpg_data = pd.read_csv('mpg.csv', encoding='utf-8')
    print("✅ MPG 데이터를 성공적으로 로드했습니다!")
    
except FileNotFoundError:
    print("❌ mpg.csv 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    # 대안으로 샘플 데이터 생성
    print("⚠️ 샘플 MPG 데이터를 생성합니다...")
    
    # 샘플 데이터 생성
    sample_data = {
        'manufacturer': ['audi', 'audi', 'audi', 'chevrolet', 'dodge'] * 20,
        'model': ['a4', 'a4', 'a4 quattro', 'malibu', 'caravan 2wd'] * 20,
        'displ': [1.8, 2.0, 2.8, 2.4, 3.3] * 20,
        'year': [1999, 2008, 1999, 2008, 1999] * 20,
        'cyl': [4, 4, 6, 4, 6] * 20,
        'trans': ['auto(l5)', 'manual(m6)', 'auto(l5)', 'auto(l4)', 'auto(l4)'] * 20,
        'drv': ['f', 'f', '4', 'f', 'f'] * 20,
        'cty': [18, 20, 16, 22, 17] * 20,
        'hwy': [29, 31, 26, 30, 24] * 20,
        'fl': ['p', 'p', 'p', 'r', 'r'] * 20,
        'category': ['compact', 'compact', 'compact', 'midsize', 'minivan'] * 20
    }
    mpg_data = pd.DataFrame(sample_data)
    
except Exception as e:
    print(f"❌ 데이터 로드 중 오류 발생: {e}")
    raise

print(f"✅ MPG 자동차 연비 데이터 준비 완료!")
print(f"📏 데이터 크기: {mpg_data.shape}")
print(f"📊 컬럼 목록: {list(mpg_data.columns)}")

# 기본 통계 정보
if 'cty' in mpg_data.columns:
    print(f"🏙️ 시내 연비 범위: {mpg_data['cty'].min():.1f} ~ {mpg_data['cty'].max():.1f} mpg")
if 'hwy' in mpg_data.columns:
    print(f"🛣️ 고속도로 연비 범위: {mpg_data['hwy'].min():.1f} ~ {mpg_data['hwy'].max():.1f} mpg")
if 'year' in mpg_data.columns:
    print(f"📅 연식 범위: {mpg_data['year'].min()} ~ {mpg_data['year'].max()}")

# 데이터 구조 확인
print(f"\n📊 데이터 구조:")
print(mpg_data.info())

print(f"\n📈 이미지와 동일한 형식으로 데이터프레임 표시:")
print("=" * 80)
# 이미지와 동일한 형식으로 첫 10개 행 표시
display_df = mpg_data.head(10).copy()
display_df.index = range(len(display_df))  # 인덱스를 0부터 시작하도록 설정
display(display_df)

print(f"\n🔍 제조사별 차량 수:")
manufacturer_counts = mpg_data['manufacturer'].value_counts()
print(manufacturer_counts)

print(f"\n🔍 차량 카테고리별 분포:")
category_counts = mpg_data['category'].value_counts()
print(category_counts)

🔄 MPG 자동차 연비 데이터를 로드하고 있습니다...
✅ MPG 데이터를 성공적으로 로드했습니다!
✅ MPG 자동차 연비 데이터 준비 완료!
📏 데이터 크기: (234, 11)
📊 컬럼 목록: ['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty', 'hwy', 'fl', 'category']
🏙️ 시내 연비 범위: 9.0 ~ 35.0 mpg
🛣️ 고속도로 연비 범위: 12.0 ~ 44.0 mpg
📅 연식 범위: 1999 ~ 2008

📊 데이터 구조:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  category      234 non-null    object 
dtypes: float64(1), int64(4), object(6

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
5,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact
6,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact
7,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
8,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact
9,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact



🔍 제조사별 차량 수:
manufacturer
dodge         37
toyota        34
volkswagen    27
ford          25
chevrolet     19
audi          18
hyundai       14
subaru        14
nissan        13
honda          9
jeep           8
pontiac        5
land rover     4
mercury        4
lincoln        3
Name: count, dtype: int64

🔍 차량 카테고리별 분포:
category
suv           62
compact       47
midsize       41
subcompact    35
pickup        33
minivan       11
2seater        5
Name: count, dtype: int64


## 📊 실습2: 데이터 전처리 및 탐색적 데이터 분석 (EDA)